In [1]:
import boto3
import botocore

import json

In [2]:
# AWS Clients

braket_config = botocore.client.Config(user_agent_extra="BraketSchemas/1.8.0")

aws_braket = boto3.client("braket", config=braket_config)
aws_s3 = boto3.client("s3")

In [3]:
# Load info for Tasks simulated on SV1

SV1_ARN = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'

response = aws_braket.search_quantum_tasks(filters=[{
    'name': 'deviceArn',
    'operator': 'EQUAL',
    'values': [SV1_ARN]
}], maxResults=25)

In [4]:
quantum_tasks = response['quantumTasks']

quantum_task = quantum_tasks[0]

quantum_task

{'createdAt': datetime.datetime(2023, 2, 28, 16, 55, 2, 235000, tzinfo=tzutc()),
 'deviceArn': 'arn:aws:braket:::device/quantum-simulator/amazon/sv1',
 'endedAt': datetime.datetime(2023, 2, 28, 16, 55, 11, 962000, tzinfo=tzutc()),
 'outputS3Bucket': 'amazon-braket-us-east-1-196332284574',
 'outputS3Directory': 'tasks/9581aea9-5357-4dba-bf3e-f97e19d5750e',
 'quantumTaskArn': 'arn:aws:braket:us-east-1:196332284574:quantum-task/9581aea9-5357-4dba-bf3e-f97e19d5750e',
 'shots': 1000,
 'status': 'COMPLETED',
 'tags': {}}

In [5]:
s3_bucket_name = quantum_task['outputS3Bucket']
s3_directory = quantum_task['outputS3Directory']
    
s3_bucket_name, s3_directory

('amazon-braket-us-east-1-196332284574',
 'tasks/9581aea9-5357-4dba-bf3e-f97e19d5750e')

In [6]:
# Download JSON

RESULTS_JSON_FILENAME = 'results-downloaded.json'

aws_s3.download_file(Bucket=s3_bucket_name,
                     Key=s3_directory + '/results.json',
                     Filename=RESULTS_JSON_FILENAME)

In [7]:
with open(RESULTS_JSON_FILENAME) as json_file:
    
    result_dict = json.load(json_file)
    

In [9]:
# result_dict['measurements'][0]

### AWS S3 Examples

In [27]:
aws_s3.upload_file(Filename='test.txt', 
                   Bucket='test-braket-bucket', 
                   Key='test-remote.txt')

In [28]:
aws_s3.download_file(Bucket='test-braket-bucket',
                     Key='test-remote.txt',
                     Filename='test-downloaded.txt')

In [30]:
with open('test-downloaded.txt') as file:

    print(file.read())

Test Task
